# Exercise 9

## Naive Bayes with Yelp review text

Using the yelp reviews database  create a Naive Bayes model to predict the star rating for reviews

Read `yelp.csv` into a DataFrame.

In [43]:
# access yelp.csv using a relative path
import pandas as pd
yelp = pd.read_csv('yelp.csv')
yelp.head(5)

,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny
0,9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,2,5,0
1,ZRJwVLyzEJq1VAihDhYiow,2011-07-27,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why some people give bad review...,review,0a2KyEL0d3Yb1V6aivbIuQ,0,0,0
2,6oRAC4uyJCsJl1X0WZpVSA,2012-06-14,IESLBzqUCLdSzSqm0eCSxQ,4,love the gyro plate. Rice is so good and I als...,review,0hT2KtfLiobPvh6cDC8JQg,0,1,0
3,_1QQZuf4zZOyFCvXc0o6Vg,2010-05-27,G-WvGaISbqqaMHlNnByodA,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",review,uZetl9T0NcROGOyFfughhg,1,2,0
4,6ozycU1RpktNG2-1BroVtw,2012-01-05,1uJFq2r5QfJG_6ExMRCaGw,5,General Manager Scott Petello is a good egg!!!...,review,vYmM4KTsC8ZfQBg-j5MWkw,0,0,0


Create a new DataFrame that only contains the 5-star and 1-star reviews.

In [44]:
# filter the DataFrame using an OR condition
yelp_best_worst = yelp[(yelp.stars==5) | (yelp.stars==1)]

Split the new DataFrame into training and testing sets, using the review text as the only feature and the star rating as the response.

In [56]:
# define X and y
X = yelp_best_worst.text
y = yelp_best_worst.stars

In [57]:
y=y.map({1:0,5:1})

In [58]:
# split into training and testing sets
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Exercise 9.1

Use CountVectorizer to create document-term matrices from X_train and X_test.

In [59]:
from sklearn.feature_extraction.text import CountVectorizer

In [60]:
vect.fit(X_train)
X_train_dtm = vect.transform(X_train)
X_train_dtm

<3064x16825 sparse matrix of type '<class 'numpy.int64'>'
	with 237720 stored elements in Compressed Sparse Row format>

In [61]:
X_test_dtm = vect.transform(X_test)
X_test_dtm

<1022x16825 sparse matrix of type '<class 'numpy.int64'>'
	with 77006 stored elements in Compressed Sparse Row format>

# Exercise 9.2

Use Naive Bayes to predict the star rating for reviews in the testing set, and calculate the accuracy.

In [62]:
# train a Naive Bayes model using X_train_dtm
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_train_dtm, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [63]:
# make class predictions for X_test_dtm
y_pred_class = nb.predict(X_test_dtm)

In [64]:
# calculate accuracy of class predictions
from sklearn import metrics
print(metrics.accuracy_score(y_test, y_pred_class))

0.918786692759


# Exercise 9.3

Calculate the AUC.

In [65]:
y_pred_prob = nb.predict_proba(X_test_dtm)[:, 1]
y_pred_prob

array([  1.00000000e+00,   9.99759874e-01,   1.00000000e+00, ...,
         1.00000000e+00,   1.05200807e-19,   9.99915316e-01])

In [66]:
# calculate AUC
print(metrics.roc_auc_score(y_test, y_pred_prob))

0.940353585141


# Exercise 9.4

Print the confusion matrix.

Calculate the Precesion, Recall and F1Score.

In [67]:
# confusion matrix
print(metrics.confusion_matrix(y_test, y_pred_class))

[[126  58]
 [ 25 813]]


In [74]:
from sklearn.metrics import precision_score, recall_score, f1_score
print('precision_score ', precision_score(y_test, y_pred_class))
print('recall_score    ', recall_score(y_test, y_pred_class))

precision_score  0.933409873708
recall_score     0.970167064439


In [75]:
print('f1_score    ', f1_score(y_test, y_pred_class))

f1_score     0.951433586893


# Exercise 9.5

Browse through the review text for some of the false positives and false negatives. Based on your knowledge of how Naive Bayes works, do you have any theories about why the model is incorrectly classifying these reviews?

In [68]:
# print message text for the false positives
X_test[y_test < y_pred_class]

2175    This has to be the worst restaurant in terms o...
1781    If you like the stuck up Scottsdale vibe this ...
2674    I'm sorry to be what seems to be the lone one ...
9984    Went last night to Whore Foods to get basics t...
3392    I found Lisa G's while driving through phoenix...
8283    Don't know where I should start. Grand opening...
2765    Went last week, and ordered a dozen variety. I...
2839    Never Again,\nI brought my Mountain Bike in (w...
321     My wife and I live around the corner, hadn't e...
1919                                         D-scust-ing.
2490    Lazy Q CLOSED in 2010.  New Owners cleaned up ...
9125    La Grande Orange Grocery has a problem. It can...
9185    For frozen yogurt quality, I give this place a...
436     this another place that i would give no stars ...
2051    Sadly with new owners comes changes on menu.  ...
1721    This is the closest to a New York hipster styl...
3447    If you want a school that cares more about you...
842     Boy is

In [98]:
X_test[8681]

'As I promised myself, I\'d go back again to try it again. Yyyeeaahhhh...sooooo....the food is still not good. I got the mongolian beef and it\'s like "wha? really? is this IRL?" \n\nI\'m a chinese food kind of a fan, this restaurant, I\'m not a fan of.'

In [69]:
# print message text for the false negatives
X_test[y_test > y_pred_class]

7148    I now consider myself an Arizonian. If you dri...
4963    This is by far my favourite department store, ...
6318    Since I have ranted recently on poor customer ...
380     This is a must try for any Mani Pedi fan. I us...
5565    I`ve had work done by this shop a few times th...
3448    I was there last week with my sisters and whil...
6050    I went to sears today to check on a layaway th...
2504    I've passed by prestige nails in walmart 100s ...
2475    This place is so great! I am a nanny and had t...
241     I was sad to come back to lai lai's and they n...
3149    I was told to see Greg after a local shop diag...
423     These guys helped me out with my rear windshie...
763     Here's the deal. I said I was done with OT, bu...
8956    I took my computer to RedSeven recently when m...
750     This store has the most pleasant employees of ...
9765    You can't give anything less than 5 stars to a...
6334    I came here today for a manicure and pedicure....
1282    Loved 

In [96]:
X_test[1266]

'I\'ve never been to this location before. My husband had taken his car here previously, and asked me to take it in today (there was a nail a front tire). The second I stepped through the doorway I was greeted, even though all agents were busy helping other customers -- excellent customer service to acknowledge I\'m there! \n\nLess than a minute later I had a gentleman walk me out to the car and look at it. He tested to see how far in the nail was, and realized it didn\'t puncture, so he took it out and said the tire was fine. \n\nI went there knowing my husband is WAY overdue for new tires, so asked for a quote. He gave me a few options, and told me what driving habits they\'d be best for. He also gave me the warranty information without me asking for it, and included it in the price instead of "shockingly" adding it in when I would pay like most places would. I never thought for a second that I was being oversold. It was amazing because I despise going anywhere having to do with fixi

Considero que uno de los motivos por los cuales el modelo podria estar clasificando mal a algunos centros de servicio (restaurantes, tiendas, locales, entre otros) podria ser por el uso frecuente o combinado de palabras (nunca, peor, mala, etc) que clasificarian de mala manera al centro de servicio cuando este comentario en realidad no lo hace (falso negativo), como en la critica 1266 en donde el cliente menciona que nunaca habia estado en este lugar pero al final recomienda este sitio y lo califica con 5 estrellas. 

Lo mismo podria estar sucediendo en el caso de los falsos positivos, en donde el uso de terminos como fan, volver, bueno, etc. podria hacer que el modelo realizara una mala prediccion en cuanto a la calidad del establecimiento, un caso puntual de este caso se observa en el texto del cliente 8681, en donde se promete a sis mismo no volver a este lugar y declara no ser fan de este restaurante. 

# Bonus Exercise 9.6 (3 points)

Let's see how well Naive Bayes performs when all reviews are included, rather than just 1-star and 5-star reviews:

- Define X and y using the original DataFrame from step 1. (y should contain 5 different classes.)
- Split the data into training and testing sets.
- Calculate the testing accuracy of a Naive Bayes model.
- Compare the testing accuracy with the null accuracy.
- Print the confusion matrix.
- Comment on the results.

In [117]:
# access yelp.csv using a relative path
import pandas as pd
yelp = pd.read_csv('yelp.csv')
yelp.head(5)

,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny
0,9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,2,5,0
1,ZRJwVLyzEJq1VAihDhYiow,2011-07-27,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why some people give bad review...,review,0a2KyEL0d3Yb1V6aivbIuQ,0,0,0
2,6oRAC4uyJCsJl1X0WZpVSA,2012-06-14,IESLBzqUCLdSzSqm0eCSxQ,4,love the gyro plate. Rice is so good and I als...,review,0hT2KtfLiobPvh6cDC8JQg,0,1,0
3,_1QQZuf4zZOyFCvXc0o6Vg,2010-05-27,G-WvGaISbqqaMHlNnByodA,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",review,uZetl9T0NcROGOyFfughhg,1,2,0
4,6ozycU1RpktNG2-1BroVtw,2012-01-05,1uJFq2r5QfJG_6ExMRCaGw,5,General Manager Scott Petello is a good egg!!!...,review,vYmM4KTsC8ZfQBg-j5MWkw,0,0,0


In [118]:
from sklearn.feature_extraction.text import CountVectorizer

In [122]:
# define X and y
X = yelp.text
y = yelp.stars

In [123]:
# split into training and testing sets
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [124]:
vect.fit(X_train)
X_train_dtm = vect.transform(X_train)
X_train_dtm

<7500x25797 sparse matrix of type '<class 'numpy.int64'>'
	with 622700 stored elements in Compressed Sparse Row format>

In [125]:
X_test_dtm = vect.transform(X_test)
X_test_dtm

<2500x25797 sparse matrix of type '<class 'numpy.int64'>'
	with 200729 stored elements in Compressed Sparse Row format>

In [126]:
# train a Naive Bayes model using X_train_dtm
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_train_dtm, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [127]:
# make class predictions for X_test_dtm
y_pred_class = nb.predict(X_test_dtm)

In [128]:
# calculate accuracy of class predictions
from sklearn import metrics
print(metrics.accuracy_score(y_test, y_pred_class))

0.4712


Se puede observar que bajo el modelo que tiene en cuenta las 5 posibles categorias (1, 2, 3, 4 o 5 estrellas) de los establecimientos de servicios existe mayor probabilidad de otorgar una clasificación erronea del establecimiento, dado que la presición del modelos fue de 0.47.  

In [132]:
# confusion matrix
print(metrics.confusion_matrix(y_test, y_pred_class))

[[ 55  14  24  65  27]
 [ 28  16  41 122  27]
 [  5   7  35 281  37]
 [  7   0  16 629 232]
 [  6   4   6 373 443]]


En la matriz de confusión podemos observar que el modelo no presenta una buena presición en el pronostico esto tambien se refleja en el accuracy de la misma, esto podria ser ocasionado por los diferentes niveles de evaluación de los establecimientos, ya que existen calificaciones intermedias para definir que tan bueno o malo es un sitio en particular, de igual manera estos niveles intermedios terminan siendo bastante subjetivos para la diferente clientela que los frecuenta.

Por lo anterior es recomendable utilizar el modelo que toma en cuenta solo dos tipos de calificaciones (1 o 5 estrellas) ya que la presición de este aumenta a 0.92, duplicando la exactitud de predicción del aterior modelo.